# Cytokine Dataset Analysis

From the project description: 

"Part of the set is a list of patients blood work measurements (cytokine levels).  These data are in a straightforward CSV file.

The other part of the dataset is genomic data of the patients.  These data are large collections of SNP markers on the patients' genomes and are represented in plink format, a common format for genome studies. 

Finally, there is a genome reference online that indicates what is known about each SNP marker.

There are about 330 patients, perhaps 40 cytokine markers, and entire genomes worth of data for each patient.  The challenge is to find patterns of SNPs (genotypes) that beget particular cytokine levels (phenotypes)."

## Prereadings
There were two papers provided as prereadings. I suggest you read both of them, this section in no way replaces the readings. I will point out some areas which I think merit special attention. 


### Genome-wide  association  study  of  survival  from  sepsis  due  to  pneumonia:  an  observational  cohort  study 
#### Introduction
Mostly motivating the analysis, no technical details. As the paper states the goal of this paper is "identifying  genetic  variations  associated  with  sepsis  survival."

#### Methods
Notice that in this paper they use 4 cohorts, we are only using the Vasopressin  in  Septic  Shock  Trial data (VASST) (cohort 2). It gives some info about the dataset "patients  with  septic  shock  were  recruited  between  July  1,  2001,  and  April  30,  2006,  and  randomly  assigned    to  receive  either  low-dose  vasopressin  or  norepinephrine." More info on the dataset specifics can be found in the other paper.

In procedures they say "We  applied  stringent  measures  of  quality  control  to  remove  unreliably  genotyped  samples  and  SNPs,  population  outliers  as  determined  by  multidimensional  scaling  of  the  genome-wide  data,  and  samples  for  which  there  were  sex  discrepancies.  The  appendix  details  the  samples  excluded  from  every  genome-wide  dataset." we are not sure if this has been done on the dataset we have, something worth considering.

More info about our dataset:
"The  number  of  autosomal  SNPs  remaining  for  imputation  were: 936  437  (VASST;  Illumina  Human  1M-Duo  BeadChip  SNP  array)."

"All  GWAS  datasets  were  imputed separately  with  IMPUTE2  and  with  1000  Genomes  Project  data  as  a  reference  panel" https://en.wikipedia.org/wiki/Imputation_(genetics)

In the diagram on page 55 pay special attention to cohort two (the one we are working with). Our dataset does not include the samples excluded due to phenotype or genotyping QC.

#### Results
"the  most  significant  association  with  28  day  survival  for  a  SNP  in  chromosome  5  (rs4957796)  in  an  intron  of  the  FER  gene" this would be a good target for your initial analysis or to confirm that your methods are consistent with the paper's findings.

"The  second  locus  that  showed  evidence  of  association  in  all  four  cohorts  (rs79423885),  although  not  achieving  even  the  less  conservative  genome-wide  significance  threshold ...  is  located  in  chromosome  6,  in  a  so-called  gene  desert  without  any  annotated  nearby  functional  elements  (appendix).  Genotyping  the  remaining  SNPs  in  the  additional  cohort  did  not  support  an  association"

"Considering  the  follow-up  of  GenOSept  and  GAinS  patients  to  6  months ...  the  effect  of  genotype  decreased  with  time,  there  being  a  significant  interaction  between  the  effect  of  genotype  and  time."
 
"The  decreased  risk  of  death  associated  with  the  C  allele  was  apparent  in  all  four  cohorts  (appendix).  Mortality  was  9·5%  in  patients  carrying  the  CC  genotype,  15·2%  in  those  carrying  the  TC  genotype,  and  25·3%  in  those  carrying  the  TT  genotype  (appendix)."

#### Discussion
"The  effect  of  the  most  significantly  associated  variant  in  FER,  rs4957796,  is  unusually  strong:  when  patients  are  stratified  based  on  genotype,  mortality  decreases  from  about  25%  in  wild-type  homozygous  (TT)  patients  to  15%  in  carriers  of  one  copy  of  the  minor  C  allele  and  is  further  reduced  to  10%  in  individuals  who  are  homozygous  for  the  C  allele."

"Although  the  most  significantly  associated  SNP  is  located  in  the  intronic  region  of  FER,  the  region  of  association  spans  several  coding  exons."

### Vasopressin  versus  Norepinephrine  Infusion  in  Patients  with  Septic  Shock
This paper is where the Vasopressin and Septic Shock Trial (VASST) cohort comes from, it contains a more in depth explanation of the origins of the dataset.
The VASST dataset had 778 patients in it however, the initial Cohort 2 VASST dataset we see being used in the previous paper had 632 patients. Possibly ask the mentor for this dataset about this discrepancy.

## Preparing the Dataset

The data is in a PLINK file format, this is typical for genetics data. We will convert to a format that is easier for analysis so you do not need to know how to work with PLINK. We are doing the conversion with [pandas-plink](https://pandas-plink.readthedocs.io/en/stable/)

In [13]:
%%capture plinkInstallOutput
# install pandas-plink to convert the plink file to something more useable
!pip install pandas-plink --user;

### PLINK files
We are going to read the PLINK files into bim, fam and G

In [14]:
from pandas_plink import read_plink
(bim, fam, G) = read_plink('RogerDonaldson-VASST2', verbose=True)

Mapping files: 100%|██████████| 3/3 [00:06<00:00,  2.98s/it]


### bim
bim is one of the PLINK files. pandas-plink has converted it to a pandas dataframe named bim 

#### Columns

chrom,     Chromosome code (aka Chromosome Number) (integer type)

snp,     [Single nucleotide polymorphisms](https://ghr.nlm.nih.gov/primer/genomicresearch/snp) (pronounced snips) 

cm,      SNP genetic position (float type)

pos,     SNP physical position (usually called bp aka base pair coordinate) (integer type)

"This file should have L lines and 4 columns, where L is
the number of SNPs contained in the dataset."
You'll notice this one has an additional a0 and a1 column (in the literature these are sometimes referred to as a1 and a2):

 a0, an allele
 
 a1, another allele

In [15]:
bim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1145510 entries, 0 to 1145509
Data columns (total 7 columns):
chrom    1145510 non-null category
snp      1145510 non-null object
cm       1145510 non-null float64
pos      1145510 non-null int64
a0       1145510 non-null category
a1       1145510 non-null category
i        1145510 non-null int64
dtypes: category(3), float64(1), int64(2), object(1)
memory usage: 38.2+ MB


In [16]:
bim.head()

,chrom,snp,cm,pos,a0,a1,i
0,1,rs12354060,0.0,10004,0,G,0
1,1,rs4345758,0.0,28663,0,A,1
2,1,rs2691310,0.0,46844,0,0,2
3,1,rs2531266,0.0,59415,0,0,3
4,1,rs4477212,0.0,72017,0,A,4


### fam

fam is one of the PLINK files. pandas-plink has converted it to a pandas dataframe named fam

fid,     Family ID (string type)

iid,     Individual ID (is it all ones? can we delete?) (string type)

father, '0' if father isn't in dataset (string type)

mother, '0' if mother isn't in dataset) (string type)

gender, '1' = male, '2' = female, '0' = unknown (integer type)

trait, the phenotype (type float)

"Columns 7 & 8 code for the observed alleles at SNP1, columns 9 &
10 code for the observed alleles at SNP2, and so on. Missing data
are coded as "0 0" as for example for SNP3 of IND1. This file should
have N lines and 2L+6 columns, where N and L are the numbers of
individuals and SNPs contained in the dataset respectively." 
You'll notice this fam file only has 6 columns (and i the index), for some reason the alleles are in the bim file rather than the fam file.

In [17]:
fam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264 entries, 0 to 1263
Data columns (total 7 columns):
fid       1264 non-null object
iid       1264 non-null object
father    1264 non-null object
mother    1264 non-null object
gender    1264 non-null category
trait     1264 non-null object
i         1264 non-null int64
dtypes: category(1), int64(1), object(5)
memory usage: 60.7+ KB


In [18]:
fam.head()

,fid,iid,father,mother,gender,trait,i
0,VB0004,1,0,0,2,2,0
1,VB0007,1,0,0,1,1,1
2,VB0010,1,0,0,1,2,2
3,VB0014,1,0,0,2,2,3
4,VB0020,1,0,0,2,1,4


### BED
Binary PED (BED) files. The PLINK binary biallelic genotype table.
To verify integrity, the first three bytes should be 0x6c, 0x1b, 0x01 in that order.
The rest of the file is a sequence of V blocks of N/4 (rounded up) bytes each, where V is the number of variants and N is the number of samples. See [here](https://www.cog-genomics.org/plink2/formats#bed) for more info on BED files.

We convert the bed PLINK file to the DASK file G. you can read more about DASK [here](https://dask.pydata.org/en/latest/docs.html). For now you can think of it like an improved pandas dataframe or numpy array (it works with the same syntax).
The i column in the bed and fam files maps to the corresponding position of the bed matrix.
"The values of the bed matrix denote how many alleles a1 (see output of data frame bim) are in the corresponding position and individual. Notice the column i in bim and fam data frames. It maps to the corresponding position of the bed matrix."

In [19]:
# Dimensions of G
G.shape
# notice how these dimensions work out perfectly for matrix multiplication with fam and bim

(1145510, 1264)

In [20]:
# how many bytes G needs (will it be too large for memory?)
G.nbytes

11583397120

In [21]:
# data type
G.dtype

dtype('int64')

Almost 12GB of data, it will probably crash if we try to load it all at once. 

In [22]:
!cat /proc/meminfo

MemTotal:       32946616 kB
MemFree:          772600 kB
MemAvailable:   10899580 kB
Buffers:            2088 kB
Cached:         10853792 kB
SwapCached:            0 kB
Active:         16052632 kB
Inactive:        3446276 kB
Active(anon):    9422004 kB
Inactive(anon):   429380 kB
Active(file):    6630628 kB
Inactive(file):  3016896 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:                 0 kB
Writeback:             0 kB
AnonPages:       8645272 kB
Mapped:           262736 kB
Shmem:           1208356 kB
Slab:            4846912 kB
SReclaimable:     732872 kB
SUnreclaim:      4114040 kB
KernelStack:       36768 kB
PageTables:       108176 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    16473308 kB
Committed_AS:   32578760 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
HardwareCorrupted:     0 kB
AnonHugePages:   

When I ran this there was almost 33GB of memory on the server but only 12GB is free, clearly fam and bim are also memory hogs. It may be different for your particular server.

To handle these hardware demands we are going to set up a simple little cluster computer.

In [23]:
%%capture daskInstallOutput
!pip install dask distributed --upgrade --user;

In [24]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:36879 Dashboard: http://127.0.0.1:41620/status,Cluster Workers: 8 Cores: 8 Memory: 33.74 GB


If anything goes wrong you can run `client.restart()` to reset the the client.

This cluster will be needed in order to run calculations on the bed file without crashing the kernel (we make the servers fairly robust so don't worry too much about breaking them). You can see the dask docs for use cases.

In [25]:
client.compute(G)

<Future: status: pending, key: finalize-cb32d4ab2c736a97497f5ab725bce5d4>

### Cytokine File
This file is apparently a csv however it is actually an excel file. Much easier to load in and work with than the PLINK files.

The column names are fairly standard and easy to google if you want to know what they represent.

It seems like the dataset is quite sparse but what is actually happening is that they did not collect information on all of the patients. Remove the patients whose values are all NaN and remove them from the PLINK dataset as well.

In [3]:
import pandas as pd
dfCytokine = pd.read_excel("RogerDonaldson-SepsisPatientCytokineLevels.xlsx")

In [5]:
dfCytokine.head()

,SampleNumber,EGF,FGF2,EOTAXIN,TGFA,GCSF,FLT3L,GMCSF,FRACTALKINE,IFNA2,...,IL1a_HW,IL8_HW,MIP1a_HW,MIP1b_HW,MMP8_HW,Elastase2_HW,Lactoferrin_HW,NGAL_HW,Resistin_HW,Thrombospondin1_HW
0,VB0004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VB0253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VB0908,9.0,21.0,63.0,2.0,33.0,3.0,11.0,8.0,15.0,...,1.64,309.74,67.63,157.83,16893.25,185785.83,424733.41,454712.34,81794.05,424193.04
3,VB0971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,VB0787,19.0,49.0,33.0,2.0,1780.0,4.0,37.0,141.0,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Using Extra Software
 If you want to install extra programs ([snptest](https://mathgen.stats.ox.ac.uk/genetics_software/snptest/snptest.html#download), [impute2](http://mathgen.stats.ox.ac.uk/impute/impute_v2.html), [metasoft](http://genetics.cs.ucla.edu/meta/), for example) to use in the notebook make sure they are compatible with the server's OS and Distro:

In [ ]:
!cat /etc/*-release 

You can access the command line for installations and other activities by putting a `!` at the beginning of the line, as seen above. You can also use the Jupyter magic `%%bash` to cover an entire cell:

In [ ]:
%%bash
cd ~/
ls

## Using PLINK Software

I already installed plink on the server. For some reason it is called "plink1" rather than "plink". The link they give to the docs is broken. Here are the actual docs: http://zzz.bwh.harvard.edu/plink/

In [1]:
!plink1 --noweb --bfile "Roger Donaldson - VASST2"


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ plink.log ]
Analysis started: Fri Jun  1 20:25:56 2018

Options in effect:
	--noweb
	--bfile Roger Donaldson - VASST2


ERROR: No file [ Roger Donaldson - VASST2.fam ] exists.
